In [1]:
# Use the Azure Machine Learning data collector to log various metrics
from azureml.logging import get_azureml_logger
logger = get_azureml_logger()

In [2]:
# Use Azure Machine Learning history magic to control history collection
# History is off by default, options are "on", "off", or "show"
%azureml history on

History logging enabled


In [3]:
from __future__ import print_function
from azure.storage.blob import BlockBlobService
import os
import cntk as C
import numpy as np
import scipy.sparse
import sys
import glob

In [4]:
#minibatch size is size of single minibatch.  To calc optimal size:
#mem = memory - neural net params * float32 size
#size = mem / (num_features * float32 size)
minibatch_size = 10000
#epoch size is virtual concept of whole data set.
epoch_size = 50000
num_features = 212

#sweep is a full pass through the data set.
max_sweeps = 1000

#num_outputs = 58
num_outputs = 1

In [5]:
# Define the task.
x = C.input_variable(num_features)
y = C.input_variable(num_outputs, is_sparse=False)
base_container = 'data-files'
train_folder = 'train_ctf'
test_folder = 'test_ctf'
base_local = '/data/bikeshare/'
model_path = os.path.join('.', 'outputs', 'model.dnn')
blob_acct_name = 'kpmgstorage1'
blob_acct_key = '2+BXi305SN45G9yyhykvp7Ij6KYka9W/WvRH4aG5fOuK+9Fenk5Yhg6X6lUMrxjpxE4wKxXyk9NYptzUpZYQkQ=='


In [6]:
final_train_file = base_local + train_folder + "/" + 'train_complete.ctf'
final_test_file = base_local + test_folder + "/" + 'test_complete.ctf'

In [7]:
final_train_file


'/data/bikeshare/train_ctf/train_complete.ctf'

In [8]:
#Download the train data into our docker container
bbs = BlockBlobService(account_name=blob_acct_name, account_key=blob_acct_key)

In [9]:
def download_and_merge_ctf(remote_base, local_base, relative, output_file_name):
    generator = bbs.list_blobs(remote_base)
    for blob in generator:
        tokens = blob.name.split('/')
        if(tokens[0] == relative
          and len(tokens) > 1
          and tokens[1] != "_SUCCESS"):
            if not os.path.exists(local_base + relative):
                os.makedirs(local_base + relative)
            bbs.get_blob_to_path(remote_base, blob.name, local_base + '/' + blob.name + '.ctf')


    read_files = glob.glob(local_base + relative + "/*.ctf")
    with open(output_file_name, "wb") as outfile:
        for f in read_files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())

In [10]:
#download_and_merge_ctf(base_container, base_local, train_folder, final_train_file)
#download_and_merge_ctf(base_container, base_local, test_folder, 'test_complete.ctf')

In [11]:
final_train_file

'/data/bikeshare/train_ctf/train_complete.ctf'

In [12]:
# Read a COMPOSITE reader to read data from both the image map and CTF files
def create_minibatch_source(ctf_file, is_training, num_outputs):
    
    # create CTF DESERIALIZER for CTF file
    data_source = C.io.CTFDeserializer(ctf_file, C.io.StreamDefs(
        features = C.io.StreamDef(field="features", shape=num_features, is_sparse=False),
        label = C.io.StreamDef(field="label", shape=num_outputs, is_sparse=False)))

    # create a minibatch source by compositing them together 
    return C.io.MinibatchSource([data_source], max_sweeps=max_sweeps, randomize=is_training)

In [13]:
train_source = create_minibatch_source(final_train_file, True, num_outputs)
#test_source = create_minibatch_source(final_test_file, False, num_outputs)

In [14]:
input_map = {    
    x : train_source.streams.features,
    y : train_source.streams.label
}


In [15]:
num_neurons = 100
def create_model(x):
    with C.layers.default_options(init = C.layers.glorot_uniform(), activation = C.relu):            
            h = C.layers.Dense(num_neurons, name="first")(x)
            h = C.layers.Dense(num_neurons, name="second")(h)
            h = C.layers.Dense(num_neurons, name="third")(h)
            h = C.layers.Dropout(dropout_rate=0.5)(h)
            p = C.layers.Dense(num_outputs, activation = None, name="prediction")(h)         
            return p

In [16]:
model = create_model(x)

In [17]:
# loss function
# loss = C.cross_entropy_with_softmax(model, y)
loss = C.squared_error(model, y)

In [18]:
#error metric function
# error = C.classification_error(model, y)
error = C.sqrt(loss)

In [19]:
#combine error and loss for training usage (AKA Criterion function)
criterion = C.combine([loss, error])

In [20]:
lr_schedule = [0.01]*100 + [0.001]*100 + [0.0001]
learner = C.sgd(model.parameters, lr_schedule, epoch_size=epoch_size)

In [21]:
#frequency is defined by number of samples
progress_writer = C.logging.ProgressPrinter(freq=epoch_size)

In [22]:
#frequence is by number of samples
checkpoint_config = C.CheckpointConfig(filename=model_path, frequency=(epoch_size * 10), restore = False)


In [23]:
#test_config = C.TestConfig(test_source)
def custom_cv_func(index, average_error, cv_num_samples, cv_num_minibatches):
    print("CV Error: " + str(average_error))
    #logger.log("Error", average_error)
    return True

In [24]:
cv_config = C.CrossValidationConfig(minibatch_source = train_source,
                                    minibatch_size = minibatch_size,
                                    model_inputs_to_streams = input_map,
                                    frequency = epoch_size,
                                    max_samples = epoch_size,
                                    callback=custom_cv_func)


In [25]:
progress = criterion.train(train_source, 
                           minibatch_size = minibatch_size,
                           model_inputs_to_streams = input_map,
                           epoch_size = epoch_size,
                           max_epochs = 100, 
                           parameter_learners = [learner], 
                           callbacks = [progress_writer, cv_config])#, checkpoint_config])#, test_config])


Learning rate per minibatch: 0.01
Finished Epoch[1]: loss = 10.167186 * 50000, metric = 239.34% * 50000 9.327s (5360.8 samples/s);
Finished Evaluation [1]: Minibatch[1-5]: metric = 192.82% * 50000;
CV Error: 1.9281672265625
Finished Epoch[2]: loss = 6.905759 * 50000, metric = 175.87% * 50000 1.328s (37650.6 samples/s);
Finished Evaluation [2]: Minibatch[1-5]: metric = 159.61% * 50000;
CV Error: 1.59610794921875
Finished Epoch[3]: loss = 5.056780 * 50000, metric = 162.62% * 50000 1.324s (37764.4 samples/s);
Finished Evaluation [3]: Minibatch[1-5]: metric = 161.32% * 50000;
CV Error: 1.61320248046875
Finished Epoch[4]: loss = 4.581927 * 50000, metric = 165.38% * 50000 1.355s (36900.4 samples/s);
Finished Evaluation [4]: Minibatch[1-5]: metric = 163.61% * 50000;
CV Error: 1.6361098046875
Finished Epoch[5]: loss = 4.540851 * 50000, metric = 166.95% * 50000 1.488s (33602.2 samples/s);
Finished Evaluation [5]: Minibatch[1-5]: metric = 165.61% * 50000;
CV Error: 1.6561012109375
Finished Epoch

Finished Epoch[44]: loss = 4.276437 * 50000, metric = 163.37% * 50000 1.993s (25087.8 samples/s);
Finished Evaluation [44]: Minibatch[1-5]: metric = 160.77% * 50000;
CV Error: 1.60773001953125
Finished Epoch[45]: loss = 4.291749 * 50000, metric = 163.14% * 50000 1.993s (25087.8 samples/s);
Finished Evaluation [45]: Minibatch[1-5]: metric = 160.79% * 50000;
CV Error: 1.60793236328125
Finished Epoch[46]: loss = 4.280090 * 50000, metric = 162.76% * 50000 1.718s (29103.6 samples/s);
Finished Evaluation [46]: Minibatch[1-5]: metric = 160.78% * 50000;
CV Error: 1.6078226953125
Finished Epoch[47]: loss = 4.297581 * 50000, metric = 163.14% * 50000 1.714s (29171.5 samples/s);
Finished Evaluation [47]: Minibatch[1-5]: metric = 160.35% * 50000;
CV Error: 1.6034521875
Finished Epoch[48]: loss = 4.277650 * 50000, metric = 162.60% * 50000 1.769s (28264.6 samples/s);
Finished Evaluation [48]: Minibatch[1-5]: metric = 161.15% * 50000;
CV Error: 1.61146208984375
Finished Epoch[49]: loss = 4.287752 * 50

Finished Epoch[87]: loss = 4.153656 * 50000, metric = 159.71% * 50000 1.646s (30376.7 samples/s);
Finished Evaluation [87]: Minibatch[1-5]: metric = 157.27% * 50000;
CV Error: 1.57269134765625
Finished Epoch[88]: loss = 4.091974 * 50000, metric = 158.88% * 50000 1.575s (31746.0 samples/s);
Finished Evaluation [88]: Minibatch[1-5]: metric = 157.97% * 50000;
CV Error: 1.57972283203125
Finished Epoch[89]: loss = 4.162017 * 50000, metric = 159.50% * 50000 1.626s (30750.3 samples/s);
Finished Evaluation [89]: Minibatch[1-5]: metric = 158.16% * 50000;
CV Error: 1.5816458984375
Finished Epoch[90]: loss = 4.140281 * 50000, metric = 159.63% * 50000 1.582s (31605.6 samples/s);
Finished Evaluation [90]: Minibatch[1-5]: metric = 158.00% * 50000;
CV Error: 1.5799973828125
Finished Epoch[91]: loss = 4.119321 * 50000, metric = 159.26% * 50000 1.715s (29154.5 samples/s);
Finished Evaluation [91]: Minibatch[1-5]: metric = 158.32% * 50000;
CV Error: 1.5831541015625
Finished Epoch[92]: loss = 4.133459 * 

In [26]:
#inf_model = C.softmax(model)

In [27]:
model.save(model_path)

In [28]:
model_path

'.\\outputs\\model.dnn'